In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

## Example ARIMA's usage

In [2]:
import pandas as pd
import numpy as np
from statsforecast.models import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsforecast.utils import AirPassengers as ap

c:\Users\jaesc2\Miniconda3\envs\skforecast\lib\site-packages\statsforecast\core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Data
# ==============================================================================
y = pd.Series(ap, index=pd.date_range('1/1/2015', periods=len(ap), freq='M'))
exog = pd.DataFrame({'exog_1': np.random.rand(len(y))}, 
                     #'exog_2': np.random.rand(len(y))},
                     index=y.index)

y_numpy = y.to_numpy()
exog_numpy = exog.to_numpy()

y_train = y.iloc[:-12]
y_test = y.iloc[-12:]
exog_train = exog.iloc[:-12]
exog_test = exog.iloc[-12:]
y_train_numpy = y_train.to_numpy()
y_test_numpy = y_test.to_numpy()
exog_train_numpy = exog_train.to_numpy()
exog_test_numpy = exog_test.to_numpy()

In [4]:
?ARIMA

Init signature:
ARIMA(
    order: Tuple[int, int, int] = (0, 0, 0),
    season_length: int = 1,
    seasonal_order: Tuple[int, int, int] = (0, 0, 0),
    include_mean: bool = True,
    include_drift: bool = False,
    include_constant: Optional[bool] = None,
    blambda: Optional[float] = None,
    biasadj: bool = False,
    method: str = 'CSS-ML',
    fixed: Optional[dict] = None,
    alias: str = 'ARIMA',
)
Docstring:     
ARIMA model.

AutoRegressive Integrated Moving Average model.

**References:**<br>
[Rob J. Hyndman, Yeasmin Khandakar (2008). "Automatic Time Series Forecasting: The forecast package for R"](https://www.jstatsoft.org/article/view/v027i03).

Parameters
----------
order : tuple (default=(0, 0, 0))
    A specification of the non-seasonal part of the ARIMA model: the three components (p, d, q) are the AR order, the degree of differencing, and the MA order.
season_length : int (default=1)
    Number of observations per unit of time. Ex: 24 Hourly data.
seasonal_order : 

In [13]:
# Model ARIMA statsforecast no exogenous variables
# ==============================================================================
arima = ARIMA(order=(1, 1, 1), seasonal_order = (0,0,0), season_length=12,
              method='CSS-ML')
arima = arima.fit(y=y_numpy)
predictions = arima.predict(h=4, level=[80])
predictions = pd.DataFrame(predictions)
predictions

,mean,lo-80,hi-80
0,480.922093,440.719279,521.124906
1,458.981630,388.938203,529.025057
2,468.821436,382.957805,554.685068
3,464.408504,363.444885,565.372122


In [14]:
# Modelo SARIMAX statsmodels no exogenous variables
# ==============================================================================
sarimax = SARIMAX(endog=y, order=(1, 1, 1), seasonal_order=(0, 0, 0, 12),
                  trend=None, ftol=2)
sarimax_res = sarimax.fit(disp=False, maxiter=1000, method='nm', ftol=1e-19)
predictions = sarimax_res.get_forecast(steps=4)
predictions.predicted_mean

c:\Users\jaesc2\Miniconda3\envs\skforecast\lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['ftol']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


2027-01-31    475.732979
2027-02-28    454.995795
2027-03-31    464.828896
2027-04-30    460.166263
Freq: M, Name: predicted_mean, dtype: float64

In [6]:
%%timeit
arima = ARIMA(order=(1, 1, 1), seasonal_order = (0,0,0), season_length=12)
arima = arima.fit(y=y_numpy)

7.46 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
sarimax = SARIMAX(endog=y, order=(1, 1, 1), seasonal_order=(0, 0, 0, 12))
sarimax_res = sarimax.fit(disp=False)

34.3 ms ± 668 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# Model ARIMA statsforecast with exogenous variables
# ==============================================================================
arima = ARIMA(order=(1, 1, 1), seasonal_order = (0,0,0), season_length=12)
arima = arima.fit(y=y_train_numpy, X=exog_train_numpy)
predictions = arima.predict(h=12, X=exog_test_numpy, level=[80])
predictions = pd.DataFrame(predictions)
predictions

,mean,lo-80,hi-80
0,439.539713,402.322332,476.757093
1,423.401344,357.527044,489.275644
2,432.171007,351.624501,512.717513
3,427.773975,332.923855,522.624096
4,430.479441,324.020506,536.938375
5,428.236883,310.976630,545.497137
6,428.692287,301.691768,555.692805
7,428.653987,292.544912,564.763061
8,427.916975,283.299889,572.534061
9,428.751053,276.086987,581.415119


In [9]:
# Model ARIMA statsmodels with exogenous variables
# ==============================================================================
sarimax = SARIMAX(endog=y_train, exog=exog_train, order=(1, 1, 1), seasonal_order=(0, 0, 0, 12))
sarimax_res = sarimax.fit(disp=False)
predictions = sarimax_res.get_forecast(steps=12, exog=exog_test, return_conf_int=False, alpha=0.2)
predictions = pd.concat((
                        predictions.predicted_mean.rename("pred"),
                        predictions.conf_int(alpha=0.10)),
                        axis = 1
                     )
predictions

c:\Users\jaesc2\Miniconda3\envs\skforecast\lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['return_conf_int', 'alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


,pred,lower y,upper y
2026-01-31,434.070211,387.217332,480.923091
2026-02-28,417.941042,338.114842,497.767242
2026-03-31,428.370613,331.590723,525.150503
2026-04-30,422.347431,308.348526,536.346336
2026-05-31,426.383329,298.855174,553.911484
2026-06-30,423.078282,282.613189,563.543375
2026-07-31,424.048889,272.109454,575.988323
2026-08-31,423.728337,260.931279,586.525395
2026-09-30,423.015167,250.141060,595.889273
2026-10-31,423.912073,241.463822,606.360325


In [66]:
# ARIMA Forward
y = np.arange(100, dtype='float')
y_new = np.arange(110, dtype='float')

exog_past = y.reshape(-1,1) # Must be floats
exog_future = y_new.reshape(-1,1) # Must be floats

arima = ARIMA(order=(2, 0, 0), season_length=12)
arima = arima.fit(y=y)

y_hat_dict = arima.forward(
                y=y_new,
                h=5, 
                level=None,
                fitted=False
            )

y_hat_dict

{'mean': array([110., 111., 112., 113., 114.])}

In [73]:
# ARIMA Forward using all series
y = np.arange(100, dtype='float')
y_new = np.arange(110, dtype='float')

exog_past = y.reshape(-1,1) # Must be floats
exog_past_new = y_new.reshape(-1,1) # Must be floats
exog_future = y_new[-5:].reshape(-1,1) # Must be floats

arima = ARIMA(order=(2, 2, 0), season_length=12)
arima = arima.fit(y=y, X=exog_past)

y_hat_dict = arima.forward(
                y=y_new,
                h=4, 
                level=None, 
                X=exog_past_new, 
                X_future=exog_future,
                fitted=False
            )

y_hat_dict

c:\Users\jaesc2\Miniconda3\envs\skforecast\lib\site-packages\statsforecast\arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)


{'mean': array([110., 111., 112., 113., 114.])}

## ForecasterSarimax

In [16]:
################################################################################
#                            ForecasterSarimax                                 #
#                                                                              #
# This work by Joaquin Amat Rodrigo and Javier Escobar Ortiz is licensed       #
# under the BSD 3-Clause License.                                              #
################################################################################
# coding=utf-8

from typing import Union, Dict, List, Tuple, Any, Optional
import warnings
import logging
import sys
import numpy as np
import pandas as pd
import statsforecast
from statsforecast.models import ARIMA
from sklearn.base import clone
from sklearn.exceptions import NotFittedError

import skforecast
from skforecast.utils import check_select_fit_kwargs
from skforecast.utils import check_y
from skforecast.utils import check_exog
from skforecast.utils import check_predict_input
from skforecast.utils import expand_index
from skforecast.utils import transform_series
from skforecast.utils import transform_dataframe

logging.basicConfig(
    format = '%(name)-10s %(levelname)-5s %(message)s', 
    level  = logging.INFO,
)


class ForecasterSarimax():
    """    
    """
    
    def __init__(
        self,
        regressor: statsforecast.models.ARIMA,
        transformer_y: Optional[object]=None,
        transformer_exog: Optional[object]=None,
        fit_kwargs: Optional[dict]=None,
        forecaster_id: Optional[Union[str, int]]=None
    ) -> None:
        
        self.regressor         = regressor
        self.transformer_y     = transformer_y
        self.transformer_exog  = transformer_exog
        self.window_size       = 1
        self.last_window       = None
        self.extended_index    = None
        self.fitted            = False
        self.index_type        = None
        self.index_freq        = None
        self.training_range    = None
        self.included_exog     = False
        self.exog_type         = None
        self.exog_col_names    = None
        self.creation_date     = pd.Timestamp.today().strftime('%Y-%m-%d %H:%M:%S')
        self.fit_date          = None
        self.skforcast_version = skforecast.__version__
        self.python_version    = sys.version.split(" ")[0]
        self.forecaster_id     = forecaster_id
        
        if not isinstance(self.regressor, ARIMA):
            raise TypeError(
                (f"`regressor` must be an instance of type statsforecast.models.ARIMA. "
                 f"Got {type(regressor)}.")
            )

        self.params = self.regressor.get_params(deep=True)

        self.fit_kwargs = check_select_fit_kwargs(
                              regressor  = regressor,
                              fit_kwargs = fit_kwargs
                          )


    def __repr__(
        self
    ) -> str:
        """
        Information displayed when a ForecasterSarimax object is printed.
        """

        info = (
            f"{'=' * len(type(self).__name__)} \n"
            f"{type(self).__name__} \n"
            f"{'=' * len(type(self).__name__)} \n"
            f"Regressor: {self.regressor} \n"
            f"Regressor parameters: {self.params} \n"
            f"fit_kwargs: {self.fit_kwargs} \n"
            f"Window size: {self.window_size} \n"
            f"Transformer for y: {self.transformer_y} \n"
            f"Transformer for exog: {self.transformer_exog} \n"
            f"Exogenous included: {self.included_exog} \n"
            f"Type of exogenous variable: {self.exog_type} \n"
            f"Exogenous variables names: {self.exog_col_names} \n"
            f"Training range: {self.training_range.to_list() if self.fitted else None} \n"
            f"Training index type: {str(self.index_type).split('.')[-1][:-2] if self.fitted else None} \n"
            f"Training index frequency: {self.index_freq if self.fitted else None} \n"
            f"Creation date: {self.creation_date} \n"
            f"Last fit date: {self.fit_date} \n"
            f"Index seen by the forecaster: {self.extended_index} \n"
            f"Skforecast version: {self.skforcast_version} \n"
            f"Python version: {self.python_version} \n"
            f"Forecaster id: {self.forecaster_id} \n"
        )

        return info


    def fit(
        self,
        y: pd.Series,
        exog: Optional[Union[pd.Series, pd.DataFrame]]=None
    ) -> None:
        """        
        """

        check_y(y=y)
        if exog is not None:
            if len(exog) != len(y):
                raise ValueError(
                    (f"`exog` must have same number of samples as `y`. "
                     f"length `exog`: ({len(exog)}), length `y`: ({len(y)})")
                )
            check_exog(exog=exog)

        # Reset values in case the forecaster has already been fitted.
        self.index_type          = None
        self.index_freq          = None
        self.last_window         = None
        self.extended_index      = None
        self.included_exog       = False
        self.exog_type           = None
        self.exog_col_names      = None
        self.X_train_col_names   = None
        self.in_sample_residuals = None
        self.fitted              = False
        self.training_range      = None
        
        if exog is not None:
            self.included_exog = True
            self.exog_type = type(exog)
            self.exog_col_names = \
                 exog.columns.to_list() if isinstance(exog, pd.DataFrame) else exog.name

        y = transform_series(
                series            = y,
                transformer       = self.transformer_y,
                fit               = True,
                inverse_transform = False
            )

        if exog is not None:
            if isinstance(exog, pd.Series):
                # pmdarima.arima.ARIMA only accepts DataFrames or 2d-arrays as exog   
                exog = exog.to_frame()
            
            exog = transform_dataframe(
                       df                = exog,
                       transformer       = self.transformer_exog,
                       fit               = True,
                       inverse_transform = False
                   )
        
        self.regressor.fit(y=y, X=exog, **self.fit_kwargs)
        self.fitted = True
        self.fit_date = pd.Timestamp.today().strftime('%Y-%m-%d %H:%M:%S')
        self.training_range = y.index[[0, -1]]
        self.index_type = type(y.index)
        if isinstance(y.index, pd.DatetimeIndex):
            self.index_freq = y.index.freqstr
        else: 
            self.index_freq = y.index.step

        self.last_window = y.copy()
        self.extended_index = self.regressor.arima_res_.fittedvalues.index.copy()
        self.params = self.regressor.get_params(deep=True)


    def predict(
        self,
        steps: int,
        last_window: Optional[pd.Series]=None,
        last_window_exog: Optional[Union[pd.Series, pd.DataFrame]]=None,
        exog: Optional[Union[pd.Series, pd.DataFrame]]=None
    ) -> pd.Series:
        """        
        """

        # Needs to be a new variable to avoid arima_res_.append if not needed
        last_window_check = last_window.copy() if last_window is not None else self.last_window.copy()

        check_predict_input(
            forecaster_name  = type(self).__name__,
            steps            = steps,
            fitted           = self.fitted,
            included_exog    = self.included_exog,
            index_type       = self.index_type,
            index_freq       = self.index_freq,
            window_size      = self.window_size,
            last_window      = last_window_check,
            last_window_exog = last_window_exog,
            exog             = exog,
            exog_type        = self.exog_type,
            exog_col_names   = self.exog_col_names,
            interval         = None,
            alpha            = None,
            max_steps        = None,
            levels           = None,
            series_col_names = None
        )

        # If last_window_exog is provided but no last_window
        if last_window is None and last_window_exog is not None:
            raise ValueError(
                ("To make predictions unrelated to the original data, both "
                 "`last_window` and `last_window_exog` must be provided.")
            )

        # Check if forecaster needs exog
        if last_window is not None and last_window_exog is None and self.included_exog:
            raise ValueError(
                ("Forecaster trained with exogenous variable/s. To make predictions "
                 "unrelated to the original data, same variable/s must be provided "
                 "using `last_window_exog`.")
            )

        if last_window is not None:
            # If predictions do not follow directly from the end of the training 
            # data. The internal statsmodels SARIMAX model needs to be updated 
            # using its append method. The data needs to start at the end of the 
            # training series.

            # check index append values
            expected_index = expand_index(index=self.extended_index, steps=1)[0]
            if expected_index != last_window.index[0]:
                raise ValueError(
                    (f"To make predictions unrelated to the original data, `last_window` "
                     f"has to start at the end of the index seen by the forecaster.\n"
                     f"    Series last index         : {self.extended_index[-1]}.\n"
                     f"    Expected index            : {expected_index}.\n"
                     f"    `last_window` index start : {last_window.index[0]}.")
                )
            
            last_window = transform_series(
                              series            = last_window.copy(),
                              transformer       = self.transformer_y,
                              fit               = False,
                              inverse_transform = False
                          )
            
            # TODO -----------------------------------------------------------------------------------------------------
            # This is done because pmdarima deletes the series name
            # Check issue: https://github.com/alkaline-ml/pmdarima/issues/535
            last_window.name = None
            # ----------------------------------------------------------------------------------------------------------

            # last_window_exog
            if last_window_exog is not None:
                # check index last_window_exog
                if expected_index != last_window_exog.index[0]:
                    raise ValueError(
                        (f"To make predictions unrelated to the original data, `last_window_exog` "
                         f"has to start at the end of the index seen by the forecaster.\n"
                         f"    Series last index              : {self.extended_index[-1]}.\n"
                         f"    Expected index                 : {expected_index}.\n"
                         f"    `last_window_exog` index start : {last_window_exog.index[0]}.")
                    )

                if isinstance(last_window_exog, pd.Series):
                    # pmdarima.arima.ARIMA only accepts DataFrames or 2d-arrays as exog 
                    last_window_exog = last_window_exog.to_frame()
            
                last_window_exog = transform_dataframe(
                                       df                = last_window_exog,
                                       transformer       = self.transformer_exog,
                                       fit               = False,
                                       inverse_transform = False
                                   )

            self.regressor.arima_res_ = self.regressor.arima_res_.append(
                                            endog = last_window,
                                            exog  = last_window_exog,
                                            refit = False
                                        )
            self.extended_index = self.regressor.arima_res_.fittedvalues.index
                        
        # Exog
        if exog is not None:
            if isinstance(exog, pd.Series):
                # pmdarima.arima.ARIMA only accepts DataFrames or 2d-arrays as exog
                exog = exog.to_frame()

            exog = transform_dataframe(
                       df                = exog,
                       transformer       = self.transformer_exog,
                       fit               = False,
                       inverse_transform = False
                   )  
            exog = exog.iloc[:steps, ]

        # Get following n steps predictions
        predictions = self.regressor.predict(
                          n_periods = steps,
                          X         = exog
                      )

        # Reverse the transformation if needed
        predictions = transform_series(
                          series            = predictions,
                          transformer       = self.transformer_y,
                          fit               = False,
                          inverse_transform = True
                      )
        predictions.name = 'pred'
        
        return predictions
    
        
    def predict_interval(
        self,
        steps: int,
        last_window: Optional[pd.Series]=None,
        last_window_exog: Optional[Union[pd.Series, pd.DataFrame]]=None,
        exog: Optional[Union[pd.Series, pd.DataFrame]]=None,
        alpha: float=0.05,
        interval: list=None,
    ) -> pd.DataFrame:
        """
        """

        # Needs to be a new variable to avoid arima_res_.append if not needed
        last_window_check = last_window.copy() if last_window is not None else self.last_window.copy()

        check_predict_input(
            forecaster_name  = type(self).__name__,
            steps            = steps,
            fitted           = self.fitted,
            included_exog    = self.included_exog,
            index_type       = self.index_type,
            index_freq       = self.index_freq,
            window_size      = self.window_size,
            last_window      = last_window_check,
            last_window_exog = last_window_exog,
            exog             = exog,
            exog_type        = self.exog_type,
            exog_col_names   = self.exog_col_names,
            interval         = interval,
            alpha            = alpha,
            max_steps        = None,
            levels           = None,
            series_col_names = None
        )

        # If last_window_exog is provided but no last_window
        if last_window is None and last_window_exog is not None:
            raise ValueError(
                ("To make predictions unrelated to the original data, both "
                 "`last_window` and `last_window_exog` must be provided.")
            )

        # Check if forecaster needs exog
        if last_window is not None and last_window_exog is None and self.included_exog:
            raise ValueError(
                ("Forecaster trained with exogenous variable/s. To make predictions "
                 "unrelated to the original data, same variable/s must be provided "
                 "using `last_window_exog`.")
            )  

        # If interval and alpha take alpha, if interval transform to alpha
        if alpha is None:
            if 100 - interval[1] != interval[0]:
                raise ValueError(
                    (f"When using `interval` in ForecasterSarimax, it must be symmetrical. "
                     f"For example, interval of 95% should be as `interval = [2.5, 97.5]`. "
                     f"Got {interval}.")
                )
            alpha = 2*(100 - interval[1])/100

        if last_window is not None:
            # If predictions do not follow directly from the end of the training 
            # data. The internal statsmodels SARIMAX model needs to be updated 
            # using its append method. The data needs to start at the end of the 
            # training series.

            # check index append values
            expected_index = expand_index(index=self.extended_index, steps=1)[0]
            if expected_index != last_window.index[0]:
                raise ValueError(
                    (f"To make predictions unrelated to the original data, `last_window` "
                     f"has to start at the end of the index seen by the forecaster.\n"
                     f"    Series last index         : {self.extended_index[-1]}.\n"
                     f"    Expected index            : {expected_index}.\n"
                     f"    `last_window` index start : {last_window.index[0]}.")
                )

            last_window = transform_series(
                              series            = last_window,
                              transformer       = self.transformer_y,
                              fit               = False,
                              inverse_transform = False
                          )

            # TODO -----------------------------------------------------------------------------------------------------
            # This is done because pmdarima deletes the series name
            # Check issue: https://github.com/alkaline-ml/pmdarima/issues/535
            last_window.name = None
            # ----------------------------------------------------------------------------------------------------------

            # Transform last_window_exog    
            if last_window_exog is not None:
                # check index last_window_exog
                if expected_index != last_window_exog.index[0]:
                    raise ValueError(
                        (f"To make predictions unrelated to the original data, `last_window_exog` "
                         f"has to start at the end of the index seen by the forecaster.\n"
                         f"    Series last index              : {self.extended_index[-1]}.\n"
                         f"    Expected index                 : {expected_index}.\n"
                         f"    `last_window_exog` index start : {last_window_exog.index[0]}.")
                    )

                if isinstance(last_window_exog, pd.Series):
                    # pmdarima.arima.ARIMA only accepts DataFrames or 2d-arrays as exog 
                    last_window_exog = last_window_exog.to_frame()
            
                last_window_exog = transform_dataframe(
                                       df                = last_window_exog,
                                       transformer       = self.transformer_exog,
                                       fit               = False,
                                       inverse_transform = False
                                   )

            self.regressor.arima_res_ = self.regressor.arima_res_.append(
                                            endog = last_window,
                                            exog  = last_window_exog,
                                            refit = False
                                        )
            self.extended_index = self.regressor.arima_res_.fittedvalues.index

        # Exog
        if exog is not None:
            if isinstance(exog, pd.Series):
                # pmdarima.arima.ARIMA only accepts DataFrames or 2d-arrays as exog
                exog = exog.to_frame()

            exog = transform_dataframe(
                       df                = exog,
                       transformer       = self.transformer_exog,
                       fit               = False,
                       inverse_transform = False
                   )  
            exog = exog.iloc[:steps, ]

        # Get following n steps predictions with intervals
        predicted_mean, conf_int = self.regressor.predict(
                                       n_periods       = steps,
                                       X               = exog,
                                       alpha           = alpha,
                                       return_conf_int = True
                                   )
                                    
        predictions = predicted_mean.to_frame(name="pred")
        predictions['lower_bound'] = conf_int[:, 0]
        predictions['upper_bound'] = conf_int[:, 1]

        # Reverse the transformation if needed
        if self.transformer_y:
            for col in predictions.columns:
                predictions[col] = transform_series(
                                    series            = predictions[col],
                                    transformer       = self.transformer_y,
                                    fit               = False,
                                    inverse_transform = True
                               )

        return predictions

    
    def set_params(
        self, 
        params: dict
    ) -> None:
        """
        
        """

        self.regressor = clone(self.regressor)
        self.regressor.set_params(**params)
        self.params = self.regressor.get_params(deep=True)


    def set_fit_kwargs(
        self, 
        fit_kwargs: dict
    ) -> None:
        """
        
        """

        self.fit_kwargs = check_select_fit_kwargs(self.regressor, fit_kwargs=fit_kwargs)


    def get_feature_importances(
        self
    ) -> pd.DataFrame:
        """

        """

        if not self.fitted:
            raise NotFittedError(
                ("This forecaster is not fitted yet. Call `fit` with appropriate "
                 "arguments before using `get_feature_importances()`.")
            )

        feature_importances = self.regressor.params().to_frame().reset_index()
        feature_importances.columns = ['feature', 'importance']

        return feature_importances


ARIMA(2,1,2)(1,0,1)[4]                   :1377.659231714479

ARIMA(0,1,0)                   :1413.9376889021364

ARIMA(1,1,0)(1,0,0)[4]                   :1393.4868889387851

ARIMA(0,1,1)(0,0,1)[4]                   :1384.0876420838147

ARIMA(2,1,2)(0,0,1)[4]                   :1376.0757744367143

ARIMA(2,1,2)                   :inf

ARIMA(2,1,2)(0,0,2)[4]                   :1361.406609163922

ARIMA(2,1,2)(1,0,2)[4]                   :1351.1200954390154

ARIMA(2,1,2)(2,0,2)[4]                   :inf

ARIMA(2,1,2)(2,0,1)[4]                   :inf

ARIMA(1,1,2)(1,0,2)[4]                   :1349.9018881777465

ARIMA(1,1,2)(0,0,2)[4]                   :1359.2954630478566

ARIMA(1,1,2)(1,0,1)[4]                   :1376.7041387051815

ARIMA(1,1,2)(2,0,2)[4]                   :inf

ARIMA(1,1,2)(0,0,1)[4]                   :1375.2559307678584

ARIMA(1,1,2)(2,0,1)[4]                   :1331.5908911378679

ARIMA(1,1,2)(2,0,0)[4]                   :1358.3302254276778

ARIMA(1,1,2)(1,0,0)[4]     

{'mean': array([497.95290608, 486.78069087, 500.0821472 , 494.10983682]),
 'lo-80': 0    467.167465
 1    442.112230
 2    450.786732
 3    440.963290
 Name: 80%, dtype: float64,
 'hi-80': 0    528.738348
 1    531.449152
 2    549.377562
 3    547.256383
 Name: 80%, dtype: float64}

In [5]:
type(arima)

statsforecast.models.AutoARIMA

In [17]:
arima.trace

True

In [18]:
arima.model_

{'coef': {'ma1': 0.05132607202710004,
  'ma2': -0.3740235946352855,
  'ma3': -0.03211229968496002,
  'sar1': -1.180628526028765,
  'sar2': -0.6920800617394647,
  'sma1': 0.7592643877288705},
 'sigma2': 577.0568477263539,
 'var_coef': array([[4.89021468e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 4.89021468e-05, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 4.89021468e-05, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.89021468e-05,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.70656778e-05,
         1.32937166e-05, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 4.89021468e-05]]),
 'mask': array([ True,  True,  True,  True,  True,  True]),
 'loglik': -657

In [19]:
from statsforecast.arima import search_arima

In [21]:
search_arima(ap)

c:\Users\jaesc2\Miniconda3\envs\skforecast\lib\site-packages\statsforecast\arima.py:1107: RuntimeWarning: invalid value encountered in sqrt
  return any(np.isnan(np.sqrt(np.diag(obj["var_coef"]))))
c:\Users\jaesc2\Miniconda3\envs\skforecast\lib\site-packages\statsforecast\arima.py:1107: RuntimeWarning: invalid value encountered in sqrt
  return any(np.isnan(np.sqrt(np.diag(obj["var_coef"]))))
c:\Users\jaesc2\Miniconda3\envs\skforecast\lib\site-packages\statsforecast\arima.py:1107: RuntimeWarning: invalid value encountered in sqrt
  return any(np.isnan(np.sqrt(np.diag(obj["var_coef"]))))


{'coef': {'ar1': 0.32216629395785024,
  'ar2': -0.21293241237138466,
  'ar3': -0.06782357715706885,
  'ar4': -0.24463522177570682,
  'sar1': 0.9842082485496031,
  'intercept': 290.6261692194776},
 'sigma2': 935.3664937984207,
 'var_coef': array([[ 4.14598157e-03,  1.59401788e-04,  2.03442820e-03,
          8.47546471e-05,  2.42037581e-04, -5.55253256e-01],
        [ 5.90754659e-10,  1.63108429e-11,  3.00046781e-10,
          1.26554334e-11,  3.53832788e-11, -8.04781566e-08],
        [ 1.56149450e-03, -2.56224527e-05,  9.02837300e-04,
          3.60738669e-05,  9.96353395e-05, -2.22167807e-01],
        [ 1.51337667e-04,  3.77844110e-05,  1.09853053e-05,
          7.27165332e-05,  2.73658282e-06, -1.56633224e-02],
        [ 1.00127322e-02,  1.05129014e-04,  5.38101683e-03,
          9.39019185e-05,  6.68926743e-04, -1.40878408e+00],
        [-7.19460556e-01, -1.46848817e-02, -3.73840424e-01,
         -1.47223945e-02, -4.40984991e-02,  9.89960242e+01]]),
 'mask': array([ True,  True,  Tru

## Tests model_selection

In [113]:
import re
import pytest
import platform
from pytest import approx
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from pmdarima.arima import ARIMA
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax.model_selection_sarimax import _evaluate_grid_hyperparameters_sarimax
from skforecast.model_selection_sarimax.model_selection_sarimax import grid_search_sarimax
from skforecast.model_selection_sarimax.model_selection_sarimax import random_search_sarimax


# Fixtures
from skforecast.ForecasterSarimax.tests.fixtures_ForecasterSarimax import y_datetime
from skforecast.ForecasterSarimax.tests.fixtures_ForecasterSarimax import exog_datetime

In [43]:
def my_metric(y_true, y_pred): # pragma: no cover
    """
    Callable metric
    """
    metric = ((y_true - y_pred)/len(y_true)).mean()
    
    return metric

In [121]:
forecaster = ForecasterSarimax(
                     regressor = Sarimax(order=(3, 2, 0), maxiter=1000, method='cg', disp=False)
                 )

param_distributions = {'order': [(2, 2, 0), (3, 2, 0)]}

results = random_search_sarimax(
                forecaster          = forecaster,
                y                   = y_datetime,
                param_distributions = param_distributions,
                n_iter              = 2,
                random_state        = 123,
                steps               = 3,
                refit               = False,
                metric              = 'mean_absolute_error',
                initial_train_size  = len(y_datetime)-12,
                fixed_train_size    = False,
                return_best         = False,
                verbose             = False
            )

Number of models compared: 2.


params grid:   0%|          | 0/2 [00:00<?, ?it/s]

In [122]:
results

,params,mean_absolute_error,order
1,"{'order': (3, 2, 0)}",0.153572,"(3, 2, 0)"
0,"{'order': (2, 2, 0)}",0.198534,"(2, 2, 0)"


In [123]:
results.iloc[:,1].to_numpy()

array([0.15357204, 0.19853423])

In [109]:
results.iloc[:,2].to_numpy()

array([0.0387042 , 0.04325543])

In [119]:
values = [(p,d,q) for p,d,q in zip(np.random.randint(0, high=4, size=3, dtype=int), 
                                       np.random.randint(0, high=4, size=3, dtype=int),
                                       np.random.randint(0, high=4, size=3, dtype=int))]

In [120]:
values

[(1, 2, 3), (2, 2, 1), (2, 0, 1)]